# Framework Comparison: Stop-Loss Strategy

This notebook validates **ml4t.backtest** stop-loss execution against:
- **VectorBT** 
- **Backtrader**

## Strategy
Long-only with stop-loss protection:
- **Entry**: Price crosses above 50-day SMA
- **Exit**: Stop-loss at 5% below entry OR price crosses below 50-day SMA
- **Position sizing**: 95% of equity

## Why Stop-Loss Validation Matters
Stop-loss execution is where many backtesting frameworks differ:
- Fill price on gap down
- Timing of stop evaluation
- Interaction with other exit signals

In [ ]:
# Configuration
ASSET = "IWM"  # Use IWM for more volatility (more stop triggers)
START_DATE = "2018-01-01"
END_DATE = "2022-12-31"
INITIAL_CASH = 100_000.0
MA_PERIOD = 50
STOP_LOSS_PCT = 0.05  # 5% stop loss
POSITION_PCT = 0.95

In [ ]:
from datetime import datetime
from pathlib import Path

import pandas as pd
import polars as pl

# Load ETF data
DATA_PATH = Path.home() / "Dropbox" / "ml4t" / "data" / "etfs" / "etf_universe.parquet"

df_full = pl.read_parquet(DATA_PATH)
df = (
    df_full.filter(pl.col("symbol") == ASSET)
    .filter(pl.col("timestamp") >= datetime.fromisoformat(START_DATE))
    .filter(pl.col("timestamp") <= datetime.fromisoformat(END_DATE))
    .sort("timestamp")
    .rename({"symbol": "asset"})
)

print(f"{ASSET} data: {len(df)} bars")
print(f"Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")

## 1. ml4t.backtest with Stop-Loss

In [ ]:
from ml4t.backtest import (
    BacktestAnalyzer,
    DataFeed,
    Engine,
    ExecutionMode,
    OrderSide,
    OrderType,
    Strategy,
)


class MAWithStopLoss(Strategy):
    """MA crossover with stop-loss."""

    def __init__(self, asset, ma_period, stop_loss_pct, position_pct):
        self.asset = asset
        self.ma_period = ma_period
        self.stop_loss_pct = stop_loss_pct
        self.position_pct = position_pct
        self.prices = []
        self.stop_order_id = None

    def on_data(self, timestamp, data, context, broker):
        if self.asset not in data:
            return

        close = data[self.asset]["close"]
        self.prices.append(close)

        if len(self.prices) > self.ma_period:
            self.prices.pop(0)

        if len(self.prices) < self.ma_period:
            return

        # Calculate MA
        ma = sum(self.prices) / len(self.prices)
        prev_close = self.prices[-2] if len(self.prices) > 1 else close
        prev_ma = (sum(self.prices[:-1]) + prev_close) / len(self.prices)  # Approx previous MA

        # Get position
        pos = broker.get_position(self.asset)
        has_position = pos is not None and pos.quantity > 0

        # Entry logic: price crosses above MA
        if close > ma and prev_close <= prev_ma and not has_position:
            equity = broker.get_account_value()
            qty = (equity * self.position_pct) / close

            # Submit entry order
            broker.submit_order(self.asset, qty, OrderSide.BUY)

            # Submit stop-loss order (5% below entry price)
            stop_price = close * (1 - self.stop_loss_pct)
            stop_order = broker.submit_order(
                self.asset, qty, OrderSide.SELL, order_type=OrderType.STOP, stop_price=stop_price
            )
            self.stop_order_id = stop_order.order_id if stop_order else None

        # Exit logic: price crosses below MA (also cancel stop)
        elif close < ma and prev_close >= prev_ma and has_position:
            broker.submit_order(self.asset, pos.quantity, OrderSide.SELL)

            # Cancel stop if still pending
            if self.stop_order_id:
                broker.cancel_order(self.stop_order_id)
                self.stop_order_id = None

In [ ]:
# Run ml4t.backtest
feed = DataFeed(prices_df=df)
strategy = MAWithStopLoss(
    asset=ASSET,
    ma_period=MA_PERIOD,
    stop_loss_pct=STOP_LOSS_PCT,
    position_pct=POSITION_PCT,
)

engine = Engine(
    feed=feed,
    strategy=strategy,
    initial_cash=INITIAL_CASH,
    execution_mode=ExecutionMode.NEXT_BAR,
)

ml4t_result = engine.run()

print("ml4t.backtest Results:")
print(f"  Final Value:  ${ml4t_result['final_value']:,.2f}")
print(f"  Total Return: {ml4t_result['total_return']:.4%}")
print(f"  Max Drawdown: {ml4t_result['max_drawdown_pct']:.4%}")

In [ ]:
# Analyze trades
analyzer = BacktestAnalyzer(engine)
stats = analyzer.trade_statistics()
print("\nTrade Statistics:")
print(f"  Total Trades: {stats.n_trades}")
print(f"  Win Rate: {stats.win_rate:.2%}")
print(
    f"  Profit Factor: {stats.profit_factor:.2f}" if stats.profit_factor else "  Profit Factor: N/A"
)

## 2. VectorBT with Stop-Loss

In [ ]:
try:
    import vectorbt as vbt

    VECTORBT_AVAILABLE = True
except ImportError:
    VECTORBT_AVAILABLE = False
    print("VectorBT not installed. Skipping.")

In [ ]:
if VECTORBT_AVAILABLE:
    pdf = df.to_pandas().set_index("timestamp")
    close = pdf["close"]
    open_price = pdf["open"]
    high = pdf["high"]
    low = pdf["low"]

    # Calculate MA and signals
    ma = close.rolling(MA_PERIOD).mean()

    # Entry: close crosses above MA
    entries = (close > ma) & (close.shift(1) <= ma.shift(1))
    entries = entries.shift(1).fillna(False)  # Next-bar execution

    # Exit: close crosses below MA
    exits = (close < ma) & (close.shift(1) >= ma.shift(1))
    exits = exits.shift(1).fillna(False)

    # Run with stop-loss
    vbt_portfolio = vbt.Portfolio.from_signals(
        close=close,
        entries=entries,
        exits=exits,
        init_cash=INITIAL_CASH,
        size=POSITION_PCT,
        size_type="percent",
        fees=0.0,
        slippage=0.0,
        freq="1D",
        sl_stop=STOP_LOSS_PCT,  # 5% stop-loss
        accumulate=False,
    )

    print("VectorBT Results:")
    print(f"  Final Value:  ${vbt_portfolio.final_value():,.2f}")
    print(f"  Total Return: {vbt_portfolio.total_return():.4%}")
    print(f"  Max Drawdown: {vbt_portfolio.max_drawdown():.4%}")

    vbt_final = vbt_portfolio.final_value()
else:
    vbt_final = None

## 3. Backtrader with Stop-Loss

In [ ]:
try:
    import backtrader as bt

    BACKTRADER_AVAILABLE = True
except ImportError:
    BACKTRADER_AVAILABLE = False
    print("Backtrader not installed. Skipping.")

In [ ]:
if BACKTRADER_AVAILABLE:

    class BTStopLossStrategy(bt.Strategy):
        params = (
            ("ma_period", MA_PERIOD),
            ("stop_loss_pct", STOP_LOSS_PCT),
            ("position_pct", POSITION_PCT),
        )

        def __init__(self):
            self.ma = bt.indicators.SMA(self.data.close, period=self.p.ma_period)
            self.crossover = bt.indicators.CrossOver(self.data.close, self.ma)
            self.stop_order = None

        def notify_order(self, order):
            if order.status in [order.Completed]:
                if order.isbuy():
                    # Place stop-loss after entry
                    stop_price = order.executed.price * (1 - self.p.stop_loss_pct)
                    self.stop_order = self.sell(exectype=bt.Order.Stop, price=stop_price)
                elif self.stop_order and order == self.stop_order:
                    self.stop_order = None

        def next(self):
            if not self.position:
                if self.crossover > 0:
                    size = (self.broker.getvalue() * self.p.position_pct) / self.data.close[0]
                    self.buy(size=size)
            else:
                if self.crossover < 0:
                    if self.stop_order:
                        self.cancel(self.stop_order)
                        self.stop_order = None
                    self.close()

    pdf = df.to_pandas().set_index("timestamp")
    bt_data = bt.feeds.PandasData(dataname=pdf, datetime=None)

    cerebro = bt.Cerebro()
    cerebro.addstrategy(BTStopLossStrategy)
    cerebro.adddata(bt_data)
    cerebro.broker.setcash(INITIAL_CASH)
    cerebro.broker.setcommission(commission=0.0)
    cerebro.broker.set_coo(True)

    cerebro.run()

    bt_final = cerebro.broker.getvalue()

    print("Backtrader Results:")
    print(f"  Final Value:  ${bt_final:,.2f}")
    print(f"  Total Return: {(bt_final - INITIAL_CASH) / INITIAL_CASH:.4%}")
else:
    bt_final = None

## 4. Results Comparison

In [ ]:
results = {
    "ml4t.backtest": ml4t_result["final_value"],
}

if VECTORBT_AVAILABLE:
    results["VectorBT"] = vbt_final

if BACKTRADER_AVAILABLE:
    results["Backtrader"] = bt_final

# Comparison table
comparison = pd.DataFrame(
    {
        "Framework": list(results.keys()),
        "Final Value": [f"${v:,.2f}" for v in results.values()],
        "Total Return": [f"{(v - INITIAL_CASH) / INITIAL_CASH:.4%}" for v in results.values()],
    }
)

print("\n" + "=" * 60)
print("STOP-LOSS STRATEGY COMPARISON")
print("=" * 60)
print(comparison.to_string(index=False))

In [ ]:
# Difference analysis
ml4t_val = ml4t_result["final_value"]

print("\nDifferences from ml4t.backtest:")
for name, val in results.items():
    if name != "ml4t.backtest":
        diff = val - ml4t_val
        pct_diff = diff / ml4t_val * 100
        print(f"  {name}: ${diff:+,.2f} ({pct_diff:+.4f}%)")

        # Note: Stop-loss timing can cause larger differences
        # due to gap fills and exact trigger timing
        tolerance = 5.0  # 5% tolerance for stop-loss strategies
        if abs(pct_diff) < tolerance:
            print(f"    ✅ Within {tolerance}% tolerance")
        else:
            print(f"    ⚠️ Difference > {tolerance}% (expected for stop-loss timing)")

## Understanding Stop-Loss Differences

Differences in stop-loss strategies between frameworks are **expected** due to:

1. **Gap Fills**: When price gaps below stop, frameworks differ in fill price:
   - ml4t.backtest: Fills at open (realistic)
   - VectorBT: May use close price
   
2. **Timing**: When stop is evaluated:
   - Before strategy logic vs. after
   - Same bar vs. next bar
   
3. **Order Cancellation**: Handling of stop when MA exit triggers:
   - Some frameworks process both, others cancel stop

### ml4t.backtest Stop-Loss Features

- **Realistic gap fill**: Fills at open when price gaps through stop
- **Exit-first processing**: Stops evaluated before entries
- **Configurable fill modes**: STOP_PRICE or NEXT_BAR_OPEN